# Diving into New York Times Best Sellers Data using PySpark

## Introduction

In this notebook, I will analyse the New York Times Bestselling books using data gathered from the New York Times API for Hardcover Fiction best sellers. The dataset spans from 7th June 2008 to 22nd July 2018. My goal is to explore various aspects of the bestselling books and gain insights into the authors, publishers, book longevity, author rankings, and rank analysis.

To answer the following questions, I will be using PySpark, a powerful distributed data processing framework. PySpark allows me to handle large-scale datasets and perform data manipulation, exploration, and analysis efficiently. I will leverage PySpark's functionality to extract insights from the New York Times Bestselling books dataset.

1. Bestselling Authors
Who are the authors with the most books on the list?
2. Publisher Analysis
Which publishers have the most books on the list?
3. Book Longevity
What are the books that have stayed the longest on the list?
4. Author Analysis
What is the highest rank authors listed have reached?
5. Rank Analysis
What's the average number of weeks a book that's reached number 1 stays in the top 20?

#### Data Preperation

Import relevant libraries and functions

In [0]:
from pyspark.sql.functions import col, from_unixtime, when, max, avg, min
from pyspark.sql.types import DateType

Read in the data

#### Data Cleaning and Transformation

In [0]:
df = sqlContext.read.json("/FileStore/tables/nyt2.json")

In [0]:
df.show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|           author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}|{{1212883200000}}|       Bantam| {1}|           {0}|           ODD HOURS|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}|{{1212883200000}}|Little, Brown| {2}|           {1}|            THE HOST|     

In [0]:
df.dtypes

Out[28]: [('_id', 'struct<$oid:string>'),
 ('amazon_product_url', 'string'),
 ('author', 'string'),
 ('bestsellers_date', 'struct<$date:struct<$numberLong:string>>'),
 ('description', 'string'),
 ('price', 'struct<$numberDouble:string,$numberInt:string>'),
 ('published_date', 'struct<$date:struct<$numberLong:string>>'),
 ('publisher', 'string'),
 ('rank', 'struct<$numberInt:string>'),
 ('rank_last_week', 'struct<$numberInt:string>'),
 ('title', 'string'),
 ('weeks_on_list', 'struct<$numberInt:string>')]

Flatten the nested fields

In [0]:
df = df.withColumn("_id", col("_id.$oid")) \
       .withColumn("bestsellers_date", (col("bestsellers_date.$date.$numberLong").cast("long") / 1000)) \
       .withColumn("published_date", (col("published_date.$date.$numberLong").cast("long") / 1000)) \
       .withColumn("price", when(col("price.$numberInt").isNotNull(), col("price.$numberInt").cast("integer")).otherwise(col("price.$numberDouble").cast("double"))) \
       .withColumn("rank", col("rank.$numberInt").cast("integer")) \
       .withColumn("rank_last_week", col("rank_last_week.$numberInt").cast("integer")) \
       .withColumn("weeks_on_list", col("weeks_on_list.$numberInt").cast("integer"))

Convert epoch to datetime

In [0]:
df = df.withColumn("bestsellers_date", from_unixtime(df["bestsellers_date"]).cast(DateType())) \
       .withColumn("published_date", from_unixtime(df["published_date"]).cast(DateType()))

In [0]:
df.dtypes

Out[31]: [('_id', 'string'),
 ('amazon_product_url', 'string'),
 ('author', 'string'),
 ('bestsellers_date', 'date'),
 ('description', 'string'),
 ('price', 'double'),
 ('published_date', 'date'),
 ('publisher', 'string'),
 ('rank', 'int'),
 ('rank_last_week', 'int'),
 ('title', 'string'),
 ('weeks_on_list', 'int')]

In [0]:
df.show(5)

+--------------------+--------------------+-----------------+----------------+--------------------+-----+--------------+-------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|           author|bestsellers_date|         description|price|published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+-----------------+----------------+--------------------+-----+--------------+-------------+----+--------------+--------------------+-------------+
|5b4aa4ead30890135...|http://www.amazon...|    Dean R Koontz|      2008-05-24|Odd Thomas, who c...| 27.0|    2008-06-08|       Bantam|   1|             0|           ODD HOURS|            1|
|5b4aa4ead30890135...|http://www.amazon...|  Stephenie Meyer|      2008-05-24|Aliens have taken...|25.99|    2008-06-08|Little, Brown|   2|             1|            THE HOST|            3|
|5b4aa4ead30890135...|http://www.amazon...|     Em

The id column will be dropped and the order of columns changed so that the book name is first.

In [0]:

df = df.drop("_id")


columns = df.columns
columns.remove("title")

df = df.select("title", *columns)

In [0]:
df.show(5)

+--------------------+--------------------+-----------------+----------------+--------------------+-----+--------------+-------------+----+--------------+-------------+
|               title|  amazon_product_url|           author|bestsellers_date|         description|price|published_date|    publisher|rank|rank_last_week|weeks_on_list|
+--------------------+--------------------+-----------------+----------------+--------------------+-----+--------------+-------------+----+--------------+-------------+
|           ODD HOURS|http://www.amazon...|    Dean R Koontz|      2008-05-24|Odd Thomas, who c...| 27.0|    2008-06-08|       Bantam|   1|             0|            1|
|            THE HOST|http://www.amazon...|  Stephenie Meyer|      2008-05-24|Aliens have taken...|25.99|    2008-06-08|Little, Brown|   2|             1|            3|
|LOVE THE ONE YOU'...|http://www.amazon...|     Emily Giffin|      2008-05-24|A woman's happy m...|24.95|    2008-06-08| St. Martin's|   3|             2| 

####  Bestselling Authors

Who are the authors with the most books on the best sellers list?

In [0]:
df.select("author","title").distinct().groupBy("author").count().orderBy("count", ascending=False).show()


+--------------------+-----+
|              author|count|
+--------------------+-----+
|      Danielle Steel|   40|
|        Stuart Woods|   35|
|     Debbie Macomber|   25|
|      David Baldacci|   22|
|       John Sandford|   20|
|            J D Robb|   18|
|       Iris Johansen|   18|
|           J A Jance|   17|
|  Richard Paul Evans|   16|
|    Sherrilyn Kenyon|   15|
|         Dean Koontz|   15|
|    Elin Hilderbrand|   15|
|Alexander McCall ...|   15|
|    Christine Feehan|   15|
|     Lisa Scottoline|   15|
|     Janet Evanovich|   15|
|James Patterson a...|   14|
|    Charlaine Harris|   13|
|  Jonathan Kellerman|   13|
|Douglas Preston a...|   13|
+--------------------+-----+
only showing top 20 rows



#### Publisher Analysis

Which publishers have the most books on the list?

In [0]:
df.select("publisher","title").distinct().groupBy("publisher").count().orderBy("count", ascending=False).show()

+--------------------+-----+
|           publisher|count|
+--------------------+-----+
|              Putnam|  293|
|       Little, Brown|  122|
|          Ballantine|  110|
|               Knopf|   96|
|       Grand Central|   93|
|    Simon & Schuster|   88|
|        St. Martin's|   85|
|           Delacorte|   83|
|Morrow/HarperCollins|   82|
|              Dutton|   71|
|           Doubleday|   64|
|        Random House|   60|
|              Bantam|   59|
|        St. Martin’s|   52|
|            Scribner|   52|
|              Viking|   51|
|               Atria|   51|
|              Harper|   46|
|              Morrow|   37|
|             Berkley|   37|
+--------------------+-----+
only showing top 20 rows



#### Book Longevity

What are the books that have stayed the longest on the list?

In [0]:
df.groupBy("title").agg(max("weeks_on_list").alias("num_weeks_on_list")).orderBy("num_weeks_on_list", ascending=False).show()

+--------------------+-----------------+
|               title|num_weeks_on_list|
+--------------------+-----------------+
|ALL THE LIGHT WE ...|              134|
|            THE HELP|              108|
|A DANCE WITH DRAGONS|               86|
|     THE NIGHTINGALE|               82|
|           GONE GIRL|               80|
|THE GIRL WHO KICK...|               79|
|THE GIRL ON THE T...|               79|
|       THE GOLDFINCH|               60|
|            THE HOST|               59|
|A GENTLEMAN IN MO...|               58|
|BEFORE WE WERE YOURS|               42|
|LITTLE FIRES EVER...|               41|
|THE STORY OF EDGA...|               39|
|THE UNDERGROUND R...|               37|
|   GO SET A WATCHMAN|               35|
|THE GUERNSEY LITE...|               33|
|      THE PARIS WIFE|               31|
|     THE LOST SYMBOL|               29|
|             INFERNO|               29|
|THE INVENTION OF ...|               27|
+--------------------+-----------------+
only showing top

#### Author Analysis

What is the highest rank authors listed have reached?

In [0]:
df.groupBy("author").agg(min("rank").alias("highest_rank")).orderBy("highest_rank", ascending=True).show()

+--------------------+------------+
|              author|highest_rank|
+--------------------+------------+
|         Jean M Auel|           1|
|         Dean Koontz|           1|
|       Sean Williams|           1|
|   Patricia Cornwell|           1|
|  Karen Marie Moning|           1|
|        Daniel Silva|           1|
|        Jodi Picoult|           1|
|  Jonathan Kellerman|           1|
|         Vince Flynn|           1|
|    Christine Feehan|           1|
|         J K Rowling|           1|
|        Stephen King|           1|
|Clive Cussler and...|           1|
|      Jeffrey Archer|           1|
|   Brandon Sanderson|           1|
|      Danielle Steel|           1|
|James Patterson a...|           1|
|  Laurell K Hamilton|           1|
|        John Grisham|           1|
|        Pierce Brown|           1|
+--------------------+------------+
only showing top 20 rows



####Rank Analysis

What's the average number of weeks a book that's reached number 1 stays in the top 20?

In [0]:
df.filter(df["rank"] == 1).agg(avg("weeks_on_list")).show()

+------------------+
|avg(weeks_on_list)|
+------------------+
|3.9716446124763705|
+------------------+

